In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yahooFinance

In [2]:
nvidia_df = yahooFinance.Ticker("NVDA").history(start="2023-08-01", end="2024-08-01")
nvidia_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-08-01 00:00:00-04:00,46.442119,46.881952,46.009287,46.489101,237858000,0.0,0.0
2023-08-02 00:00:00-04:00,45.813368,45.822363,43.370308,44.251968,530142000,0.0,0.0
2023-08-03 00:00:00-04:00,43.783146,45.100640,43.783146,44.497871,324176000,0.0,0.0
2023-08-04 00:00:00-04:00,44.967687,45.624431,44.375915,44.662804,362666000,0.0,0.0
2023-08-07 00:00:00-04:00,45.093643,45.522478,44.545853,45.399525,322154000,0.0,0.0


In [3]:
print(len(nvidia_df))

252


In [4]:
EVENT_DATE = pd.to_datetime("2024-05-22")
EVENT_NAME = "NVIDIA Q1 2024 Earnings Call"
EVENT_WINDOW = [2, 5] # days

def get_returns(stock_prices):
    stock_prices = stock_prices.values
    returns = []
    for i in range(1, len(stock_prices)):
        returns.append((stock_prices[i] - stock_prices[i - 1]) / stock_prices[i - 1])
    returns.append(0)
    return np.array(returns)

def get_AR_CMR_model(returns):
    """
    returns the abnormal returns and MVA abnormal returns calculated using the CMR model
    """
    avg_returns, std_returns = np.mean(returns), np.std(returns)
    abnormal_returns = returns - avg_returns
    MVA_abnormal_returns = np.convolve(abnormal_returns, np.ones(EVENT_WINDOW[1] + EVENT_WINDOW[0]) / (EVENT_WINDOW[1] + EVENT_WINDOW[0]), mode="valid")
    abnormal_returns_std = std_returns / np.sqrt(EVENT_WINDOW[1] + EVENT_WINDOW[0])
    print(f"MSE of the model: {np.mean(abnormal_returns ** 2): e}")
    return abnormal_returns, MVA_abnormal_returns, avg_returns, abnormal_returns_std

In [5]:
import plotly.graph_objects as go

nvidia_returns = get_returns(nvidia_df["Close"])
nvidia_abnormal_returns, nvidia_MVA_abnormal_returns, nvidia_avg_returns, nvidia_abnormal_returns_std = get_AR_CMR_model(nvidia_returns)

MSE of the model:  9.415009e-04


In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=nvidia_df.index, y=nvidia_abnormal_returns, mode="lines", name="Abnormal Returns"))
fig.add_trace(go.Scatter(x=nvidia_df.index, y=nvidia_MVA_abnormal_returns, mode="lines", name="MVA Abnormal Returns"))
fig.add_hrect(y0=nvidia_avg_returns-1.96*nvidia_abnormal_returns_std, y1=nvidia_avg_returns+1.96*nvidia_abnormal_returns_std, fillcolor="lightgrey", opacity=0.5, layer="below", line_width=0)

In [7]:
tsmc_df = yahooFinance.Ticker("TSM").history(start="2023-08-01", end="2024-08-01")
tsmc_returns = get_returns(tsmc_df["Close"])
tsmc_abnormal_returns, tsmc_MVA_abnormal_returns, tsmc_avg_returns, tsmc_abnormal_returns_std = get_AR_CMR_model(tsmc_returns)

fig.add_trace(go.Scatter(x=tsmc_df.index, y=tsmc_abnormal_returns, mode="lines", name="Abnormal Returns (TSM)"))
fig.add_trace(go.Scatter(x=tsmc_df.index, y=tsmc_MVA_abnormal_returns, mode="lines", name="MVA Abnormal Returns (TSM)"))
fig.add_hrect(y0=tsmc_avg_returns-1.96*tsmc_abnormal_returns_std, y1=tsmc_avg_returns+1.96*tsmc_abnormal_returns_std, fillcolor="lightgrey", opacity=0.5, layer="below", line_width=0)

MSE of the model:  4.715234e-04


In [8]:
spy_df = yahooFinance.Ticker("SPY").history(start="2023-08-01", end="2024-08-01")
spy_returns = get_returns(spy_df["Close"])
spy_abnormal_returns, spy_MVA_abnormal_returns, spy_avg_returns, spy_abnormal_returns_std = get_AR_CMR_model(spy_returns)

MSE of the model:  5.333394e-05


In [9]:
# plot all the abnormal returns MVA

fig = go.Figure()
fig.add_trace(go.Scatter(x=nvidia_df.index, y=nvidia_MVA_abnormal_returns/nvidia_avg_returns, mode="lines", name="MVA Abnormal Returns (NVDA)"))
fig.add_trace(go.Scatter(x=tsmc_df.index, y=tsmc_MVA_abnormal_returns/tsmc_avg_returns, mode="lines", name="MVA Abnormal Returns (TSM)"))
fig.add_trace(go.Scatter(x=spy_df.index, y=spy_MVA_abnormal_returns/spy_avg_returns, mode="lines", name="MVA Abnormal Returns (SPY)"))

In [10]:
# calculate abnormal returns using CAPM model

def get_AR_CAPM_model(returns, market_returns):
    """
    returns the abnormal returns using CAPM model
    """
    beta = np.cov(returns, market_returns)[0, 1] / np.var(market_returns)
    avg_market_returns = np.mean(market_returns)
    avg_returns = np.mean(returns)
    abnormal_returns = returns - (avg_market_returns + beta * (market_returns - avg_market_returns))
    MVA_abnormal_returns = np.convolve(abnormal_returns, np.ones(EVENT_WINDOW[1] + EVENT_WINDOW[0]) / (EVENT_WINDOW[1] + EVENT_WINDOW[0]), mode="valid")
    abnormal_returns_std = np.std(abnormal_returns) / np.sqrt(EVENT_WINDOW[1] + EVENT_WINDOW[0])
    print(f"MSE of the model: {np.mean(abnormal_returns ** 2): e}")
    return abnormal_returns, MVA_abnormal_returns, avg_returns, abnormal_returns_std

nvidia_abnormal_returns_CAPM, nvidia_MVA_abnormal_returns_CAPM, nvidia_avg_returns, nvidia_abnormal_returns_std = get_AR_CAPM_model(nvidia_returns, spy_returns)
tsmc_abnormal_returns_CAPM, tsmc_MVA_abnormal_returns_CAPM, tsmc_avg_returns, tsmc_abnormal_returns_std = get_AR_CAPM_model(tsmc_returns, spy_returns)

MSE of the model:  6.259554e-04
MSE of the model:  2.924430e-04


In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=nvidia_df.index, y=nvidia_MVA_abnormal_returns_CAPM, mode="lines", name="MVA Abnormal Returns (NVDA)"))
fig.add_hrect(y0=nvidia_avg_returns-1.96*nvidia_abnormal_returns_std, y1=nvidia_avg_returns+1.96*nvidia_abnormal_returns_std, fillcolor="orange", opacity=0.5, layer="below", line_width=0)

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=tsmc_df.index, y=tsmc_MVA_abnormal_returns_CAPM, mode="lines", name="MVA Abnormal Returns (TSM)"))
fig.add_hrect(y0=tsmc_avg_returns-1.96*tsmc_abnormal_returns_std, y1=tsmc_avg_returns+1.96*tsmc_abnormal_returns_std, fillcolor="orange", opacity=0.5, layer="below", line_width=0)


In [13]:
ff_coeff_df = pd.read_csv("../data/F-F_Research_Data_Factors_daily.csv")
ff_coeff_df["date"] = pd.to_datetime(ff_coeff_df["date"], format="%Y%m%d")
ff_coeff_df = ff_coeff_df[(ff_coeff_df["date"] >= "2023-08-01") & (ff_coeff_df["date"] <= "2024-08-01")]
ff_coeff_df = ff_coeff_df.set_index("date")
ff_coeff_df

,Mkt-RF,SMB,HML,RF
date,,,,
2023-08-01,-0.35,-0.17,0.30,0.020
2023-08-02,-1.49,-0.19,0.60,0.020
2023-08-03,-0.19,-0.05,0.08,0.020
2023-08-04,-0.49,0.04,0.76,0.020
2023-08-07,0.77,-1.16,0.59,0.020
...,...,...,...,...
2024-07-25,-0.30,1.49,0.67,0.021
2024-07-26,1.10,0.57,-0.19,0.021
2024-07-29,-0.02,-1.04,-0.44,0.021


In [14]:
# calculate abnormal returns using Fama-French model

def get_AR_Fama_French_model(returns, market_returns, SMB, HML):
    """
    returns the abnormal returns using Fama-French model
    """
    b1 = np.cov(returns, market_returns)[0, 1] / np.var(market_returns)
    b2 = np.cov(returns, SMB)[0, 1] / np.var(SMB)
    b3 = np.cov(returns, HML)[0, 1] / np.var(HML)
    avg_market_returns = np.mean(market_returns)
    avg_SMB = np.mean(SMB)
    avg_HML = np.mean(HML)
    avg_returns = np.mean(returns)
    abnormal_returns = returns - (avg_market_returns + b1 * (market_returns - avg_market_returns) + b2 * (SMB - avg_SMB) + b3 * (HML - avg_HML))
    MVA_abnormal_returns = np.convolve(abnormal_returns, np.ones(EVENT_WINDOW[1] + EVENT_WINDOW[0]) / (EVENT_WINDOW[1] + EVENT_WINDOW[0]), mode="valid")
    abnormal_returns_std = np.std(abnormal_returns) / np.sqrt(EVENT_WINDOW[1] + EVENT_WINDOW[0])
    print(f"MSE of the model: {np.mean(abnormal_returns ** 2): e}")
    return abnormal_returns, MVA_abnormal_returns, avg_returns, abnormal_returns_std

nvidia_abnormal_returns_FF, nvidia_MVA_abnormal_returns_FF, nvidia_avg_returns, nvidia_abnormal_returns_std = get_AR_Fama_French_model(nvidia_returns, spy_returns, ff_coeff_df["SMB"], ff_coeff_df["HML"])

fig = go.Figure()
fig.add_trace(go.Scatter(x=nvidia_df.index, y=nvidia_MVA_abnormal_returns_FF, mode="lines", name="MVA Abnormal Returns (NVDA)"))
fig.add_hrect(y0=nvidia_avg_returns-1.96*nvidia_abnormal_returns_std, y1=nvidia_avg_returns+1.96*nvidia_abnormal_returns_std, fillcolor="orange", opacity=0.5, layer="below", line_width=0)

MSE of the model:  6.196762e-04
